# Linking the functional and stimulus datasets

Now that we have a functional dataset and a mappings from stimulus to labels, we can link the two together and finalize our dataset pipeline.

In [2]:
from packages import *
%matplotlib inline

In [3]:
# run = False
run = True
# save = False
save = True
load = False
# load = True
# check = False
check = True

In [4]:
subject = 3

In [5]:
def load_events_files(subject):
    '''
    loads the events files associated with a subject in a dictionary keyed by session and run
    '''
    data = {}
    sub_dir = os.path.join(DATA_DIR, get_subject_dir(subject))
    for session in SESSIONS:
        data[session] = {}
        ses_dir = os.path.join(sub_dir, get_session_dir(session), 'events/')
        for f in os.listdir(ses_dir):
            if 'localizer' in f:
                continue
            run = extract_run(f)
            f_path = os.path.join(ses_dir, f)
            df = load_file(f_path)[0]
            # we can get rid of irrelevant columns in data
            data[session][run] = df[['ImgName']]
    return data      

In [6]:
if run:
    data = load_events_files(subject)

In [8]:
data[1][1].head(10)

,ImgName
0,COCO_train2014_000000292845.jpg
1,COCO_train2014_000000296474.jpg
2,COCO_train2014_000000340488.jpg
3,COCO_train2014_000000072095.jpg
4,COCO_train2014_000000207833.jpg
5,cabinIndoors4.jpg
6,dugout2.jpg
7,n02640242_4958.JPEG
8,n02123597_11635.JPEG
9,n04467665_69667.JPEG


Next up we will transform our ses:run dictionary into ses:stim, which is the format of our fmri dataset.

In [9]:
if run:
    dataset = {}
    sessions = data.keys()
    sessions.sort()
    for ses in sessions:
        combined = []
        runs = data[ses].keys()
        runs.sort()
        for r in runs: 
            for i in xrange(len(data[ses][r])):
                combined.append(data[ses][r].iloc[i].ImgName)
        dataset[ses] = combined

In [14]:
len(dataset[1])

370

In [38]:
if save:
    save_pickle(dataset, 'func_stim_link', subdir='sub-{}'.format(subject))
    